In [ ]:
# MUSIC GENRE CLASSIFICATION USING RNN

In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
# data preprocessing

In [ ]:
#pip install librosa

In [3]:
data_dir=r"C:\Users\sweth\Downloads\Data\genres_original"
file_paths=[]
labels=[]
for genre in os.listdir(data_dir):
    genre_folder=os.path.join(data_dir,genre)
    if os.path.isdir(genre_folder):
        for filename in os.listdir(genre_folder):
            if filename.endswith('.wav'):
                file_paths.append(os.path.join(genre_folder,filename))
                labels.append(genre)

In [5]:
import librosa
features=[]
bad_files=[]
new_labels=[] # remove the labels that correspond to any bad files, otherwise your data and labels won't match
for path,label in zip(file_paths,labels):
    try:
        y,sr=librosa.load(path,duration=30)
        mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
        mfcc=mfcc.T # Transpose to (time_steps,features)
        features.append(mfcc)
        new_labels.append(label) # only add label if feature extraction was successful
    except Exception as e:
        print(f"Error processing {path}: {e}")
        bad_files.append(path)    

C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
C:\Users\sweth\AppData\Local\Temp\ipykernel_32412\489068962.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y,sr=librosa.load(path,duration=30)
C:\Users\sweth\AppData\Roaming\Python\

Error processing C:\Users\sweth\Downloads\Data\genres_original\jazz\jazz.00054.wav: 


In [7]:
print(len(features),len(labels))
print(type(features[0]),features[0].shape)

999 1000
<class 'numpy.ndarray'> (1292, 40)


In [9]:
# Clean your features list to include only valid ones(along with their labels)
# To ensure you don't run into mismatched or corrupted inputs later
cleaned_features=[]
cleaned_labels=[]
for f,l in zip(features,labels):
    if isinstance(f,(list,np.ndarray)):
        cleaned_features.append(f)
        cleaned_labels.append(l)       

In [ ]:
# data preparation

In [11]:
# Pad sequences so that all samples have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_features=pad_sequences(cleaned_features,dtype='float32',padding='post')

In [13]:
print(len(cleaned_features),len(cleaned_labels))
print(type(cleaned_features[0]),cleaned_features[0].shape)
print(padded_features.shape)

999 999
<class 'numpy.ndarray'> (1292, 40)
(999, 1292, 40)


In [15]:
# Encoding labels-Neural networks need numerical labels...
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
encoded_labels=le.fit_transform(cleaned_labels)

In [17]:
print(le.classes_)

['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']


In [ ]:
# train test split

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(padded_features,encoded_labels,test_size=0.2,random_state=42)

In [ ]:
# Model building

In [21]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,SpatialDropout1D
from tensorflow.keras.layers import Dropout

model=Sequential()
model.add(LSTM(128,input_shape=(1292,40),return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10,activation='softmax')) 
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

C:\Users\sweth\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        86,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,434 (372.79 KB)

 Trainable params: 95,434 (372.79 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# training

In [23]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,batch_size=32)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.0903 - loss: 2.4600 - val_accuracy: 0.2100 - val_loss: 2.1973
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.2499 - loss: 2.1145 - val_accuracy: 0.2650 - val_loss: 2.0593
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.3250 - loss: 1.9683 - val_accuracy: 0.2650 - val_loss: 1.9791
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.3834 - loss: 1.8126 - val_accuracy: 0.2950 - val_loss: 1.8773
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.4056 - loss: 1.7239 - val_accuracy: 0.3200 - val_loss: 1.8007
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.4490 - loss: 1.6438 - val_accuracy: 0.3700 - val_loss: 1.7636
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.4155 - loss: 1.6340 - val_accuracy: 0.4000 - val_loss: 1.7181
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.4812 - loss: 1.4856 - val_accuracy: 0.4150 - val_loss:

In [25]:
# save model
model.save('music_genre_classifier.keras')

In [ ]:
# prediction

In [27]:
sample=r"C:\Users\sweth\Downloads\Data\genres_original\classical\classical.00075.wav"
y,sr=librosa.load(sample,duration=30)
mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
mfcc=mfcc.T 

In [29]:
padded_mfcc=pad_sequences([mfcc],dtype='float32',padding='post')

In [31]:
new_labels=np.unique(cleaned_labels)

In [33]:
new_labels

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype='<U9')

In [35]:
prediction=model.predict(padded_mfcc)
predicted_index=np.argmax(prediction)
print('Predicted Genre:',new_labels[predicted_index])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
Predicted Genre: classical


In [ ]:
# tkinter

In [ ]:
#pip install pyttsx3

In [37]:
import tkinter as tk
from tkinter import filedialog
import librosa
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your trained model and label list
model=load_model('music_genre_classifier.keras')
new_labels=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock'] 


# Prediction function
def predict_genre(file_path):
    try:
        y,sr=librosa.load(file_path,duration=30)
        mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
        mfcc=mfcc.T
        padded_mfcc=pad_sequences([mfcc],dtype='float32',padding='post') 
        prediction=model.predict(padded_mfcc)
        predicted_index=np.argmax(prediction)
        return new_labels[predicted_index]
    except Exception as e:
        return f"Error: {str(e)}"

import pyttsx3
engine=pyttsx3.init()
engine.setProperty('rate',120)
def speak(text):
    engine.say(text)
    engine.runAndWait()

# Tkinter GUI
def browse_file():
    file_path=filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
    if file_path:
        genre=predict_genre(file_path)
        result_label.config(text=f"Predicted Genre: {genre}")
        speak(f"Genre is {genre}")

root=tk.Tk()
root.title("Music Genre Classifier")
root.geometry("1000x700")

from PIL import Image,ImageTk
image=Image.open(r"C:\Users\sweth\Downloads\mgc.webp")
bg_photo=ImageTk.PhotoImage(image)
bg_label=tk.Label(root,image=bg_photo)
bg_label.place(x=0,y=0,relwidth=1,relheight=1)

l=tk.Label(root,text="MUSIC GENRE CLASSIFIER",font=("Arial",26,"bold")).pack(pady=10)
l1=tk.Label(root,text="Select an audio file from your device",font=("Arial",14,"bold")).pack(pady=10)
b=tk.Button(root,text="Upload and Predict",font=("Arial",12,"bold"),command=browse_file,bg='light grey').pack(pady=5)
result_label=tk.Label(root,text="",font=("Arial",14))
result_label.pack(pady=20)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
